# Goals:
- More corpus and questions. 
- Ideally we get different style corpuses.
1. Research:
Start with WikiText then use this to look for other text datasets.
2. Compose a list of all different text datasets.
3. Get a sample from each type.
4. Generate questions for each dataset. 


Tokens extracted from good wiki articles
https://huggingface.co/datasets/wikitext

JSON Conversations between a user and a chat bot. 
https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k?row=0

Finance:
https://huggingface.co/datasets/AdaptLLM/finance-tasks?row=12

Finance SEC fillings:
https://huggingface.co/datasets/JanosAudran/financial-reports-sec

PubMed Open Access:
https://huggingface.co/datasets/pmc/open_access

Legal:
https://huggingface.co/datasets/pile-of-law/pile-of-law



Next goal?

We have a lot of weird data. Now what. 

For each dataset, attempt to get large texts out of it. These texts will then be our 'main docs'. Our corpus dataset will be all these docs for which we run chunking over each individual, then union of chunks. Questions generated just be looking at single doc. 

Current goal:
Get 'docs' from datasets.


In [2]:
import requests

def fetch_huggingface_data(params):
    # URL for the API endpoint
    url = "https://datasets-server.huggingface.co/rows"

    # Make the GET request
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Get the JSON response
        data = response.json()
        return data
    else:
        print("Failed to fetch data:", response.status_code)
        return None

# WikiTexts

In [3]:

data = fetch_huggingface_data({
    'dataset': 'wikitext',
    'config': 'wikitext-103-v1',
    'split': 'train',
    'offset': 0,
    'length': 100
})

total_doc = ''

for row in data['rows']:
    total_doc += row['row']['text']

print(len(total_doc))
print(total_doc[:100])

35458
 = Valkyria Chronicles III = 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit
